In [21]:
import numpy as np
import pandas as pd
import math
import random
import pickle

# to make the experimens replicable
random.seed(123456)

In [2]:
df = pd.read_pickle('regione-toscana/atti-dirigenti.pkl')

In [3]:
dataset = df[['OGGETTO', 'UFFICIO_DG']]

In [4]:
dataset.head()

,OGGETTO,UFFICIO_DG
0,DGR 968/07 e s.m.i. Accreditamento degli organ...,50125
1,REG. CEE 2080/92 - AIUTI ALLE MISURE FORESTALI...,50117
2,"Servizi autorizzati ex art.14 L.R. 42/98 ""Norm...",50121
3,"L.R. 1/2006, DCR 3/2012, L.R. 1/2015 art.29, c...",50117
4,L.R. n. 87/1997 - Pubblicazione sul BURT dell'...,50116


In [5]:
dataset.shape

(153206, 2)

### Compute the number of classes

In [6]:
len(set(dataset.UFFICIO_DG))

43

### Groups the documents by office

In [7]:
documents_per_office = dataset.groupby(['UFFICIO_DG']).count()
documents_per_office.describe()

,OGGETTO
count,43.000000
mean,3562.930233
std,3408.012746
min,106.000000
25%,1020.000000
50%,2153.000000
75%,5418.000000
max,13890.000000


### We select the documents with frequency greater that the 25 percentile

In [8]:
value = documents_per_office.quantile(0.25)[0]
sel_dataset = documents_per_office[documents_per_office.OGGETTO >= value]
sel_dataset.shape

(32, 1)

In [9]:
sel_dataset['UFFICIO_DG'] = sel_dataset.index

/home/fabio/miniconda3/envs/dl-test/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Join the dataset to select a subset of the data

In [10]:
final_ds = dataset.merge(sel_dataset, how='inner', on=['UFFICIO_DG'])

In [11]:
final_ds.shape

(147463, 3)

In [12]:
dataset.shape

(153206, 2)

In [13]:
final_ds.head()

,OGGETTO_x,UFFICIO_DG,OGGETTO_y
0,DGR 968/07 e s.m.i. Accreditamento degli organ...,50125,4281
1,Nomina Commissione d’esame per il percorso for...,50125,4281
2,Nomina della Commissione d'esame matricola 201...,50125,4281
3,Progetti formativi per drop-out a.s.f. 2015-20...,50125,4281
4,REG (CE) 1080/2006-Por Creo Fesr 2007-2013-Lin...,50125,4281


### Select the OGGETTO and UFFICIO

In [14]:
final_ds = final_ds[['OGGETTO_x',"UFFICIO_DG"]]

In [15]:
final_ds.head()

,OGGETTO_x,UFFICIO_DG
0,DGR 968/07 e s.m.i. Accreditamento degli organ...,50125
1,Nomina Commissione d’esame per il percorso for...,50125
2,Nomina della Commissione d'esame matricola 201...,50125
3,Progetti formativi per drop-out a.s.f. 2015-20...,50125
4,REG (CE) 1080/2006-Por Creo Fesr 2007-2013-Lin...,50125


### Transform it in a dataset

In [16]:
samples = []
labels = []

In [17]:
for text, label in final_ds.as_matrix():
    samples.append(text)
    labels.append(label)

In [18]:
samples[:5]

["DGR 968/07 e s.m.i. Accreditamento degli organismi formativi. Rilascio dell'accreditamento all'organismo formativo Bioscience Research Center - cod. GR1035.",
 'Nomina Commissione d’esame per il percorso formativo “Formazione obbligatoria per utilizzatori professionali di prodotti fitosanitari”, MATRICOLA N. 2016SI0046',
 "Nomina della Commissione d'esame matricola 2016PI0301. Agenzia Formativa Cescot",
 'Progetti formativi per drop-out a.s.f. 2015-2016. Integrazione impegno per progetto "Figaro - Operatore del benessere (acconciatura)"',
 'REG (CE) 1080/2006-Por Creo Fesr 2007-2013-Linea di intevento 5.1.d-Chiusura attività']

In [19]:
labels[:5]

['50125', '50125', '50125', '50125', '50125']

In [20]:
samples = np.array(samples)
labels = np.array(labels)

In [23]:
with open('regione-toscana/dataset-dirigenti.pkl', 'wb') as o:
    pickle.dump((samples, labels), o)

In [24]:
dataset_path = 'regione-toscana/dataset-dirigenti.pkl'

with open(dataset_path, 'rb') as f:
    samples, labels = pickle.load(f)